<a href="https://colab.research.google.com/github/Rud896/mnitsatisfaction/blob/main/stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import PolynomialFeatures

In [46]:
ticker='VMM.NS'
stock=yf.Ticker(ticker)
historical_data = stock.history(period="max")

In [47]:
poly=PolynomialFeatures(degree=2,include_bias=False)
polyfeat=poly.fit_transform(historical_data)
historical_data=pd.DataFrame(polyfeat)
historical_data.tail()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
113,126.300003,128.899994,124.879997,126.040001,9157789.0,0.0,0.0,15951.690771,16280.069622,15772.344034,...,15886.081831,1.154248e+09,0.0,0.0,8.386510e+13,0.0,0.0,0.0,0.0,0.0
114,126.370003,132.889999,125.440002,127.790001,31868957.0,0.0,0.0,15969.377594,16793.309588,15851.853453,...,16330.284334,4.072534e+09,0.0,0.0,1.015630e+15,0.0,0.0,0.0,0.0,0.0
115,128.600006,129.360001,126.309998,127.230003,8812233.0,0.0,0.0,16537.961570,16635.696868,16243.466457,...,16187.473754,1.121180e+09,0.0,0.0,7.765545e+13,0.0,0.0,0.0,0.0,0.0
116,127.839996,134.600006,127.599998,131.009995,16881713.0,0.0,0.0,16343.064664,17207.264287,16312.383338,...,17163.618661,2.211673e+09,0.0,0.0,2.849922e+14,0.0,0.0,0.0,0.0,0.0
117,131.610001,131.979996,128.699997,130.320007,14382318.0,0.0,0.0,17321.192261,17369.887318,16938.206677,...,16983.304309,1.874304e+09,0.0,0.0,2.068511e+14,0.0,0.0,0.0,0.0,0.0


In [48]:
print(polyfeat.shape)

(118, 35)


In [49]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout

# Define the model
model = Sequential([
    # LSTM layer with 50 units
    LSTM(100, activation='relu', return_sequences=False,input_shape=(10, 35),kernel_regularizer=l2(0.001),
     recurrent_regularizer=l2(0.001)),
# Dense layer for output
    Dense(4,name='ld')  # Assuming a regression task with 1 output value
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01 ), loss='mse')

# Summary of the model architecture
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 100)            │        54,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ld (Dense)                      │ (None, 4)              │           404 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,804 (214.08 KB)

 Trainable params: 54,804 (214.08 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [51]:
features=historical_data.values
features.shape

(118, 35)

In [52]:
scaler=preprocessing.MinMaxScaler()
scaled_data=scaler.fit_transform(features)

In [53]:
timesteps = 15

X_train = []
y_train = []

for i in range(timesteps, len(scaled_data)):
    X_train.append(scaled_data[i - timesteps:i])
    y_train.append(scaled_data[i, 0:4])

X_train, y_train = np.array(X_train), np.array(y_train)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

Shape of X_train: (103, 15, 35)
Shape of y_train: (103, 4)


In [54]:
t=int(np.round(X_train.shape[0]*0.8))
xtrain,ytrain=X_train[0:t],y_train[0:t]
xtest,ytest=X_train[t:],y_train[t:]
xtest.shape
print("X_train shape:", xtrain.shape)
print("Y_train shape:", ytrain.shape)
print("X_test shape:", xtest.shape)
print("Y_test shape:", ytest.shape)

X_train shape: (82, 15, 35)
Y_train shape: (82, 4)
X_test shape: (21, 15, 35)
Y_test shape: (21, 4)


In [55]:
model.fit(xtrain, ytrain, epochs=100, batch_size=5)

Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 0.1565
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0411
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0234
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0175
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0130
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0143
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0142
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0120
Epoch 9/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0139
Epoch 10/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0122
Epoch 11/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0109
Epoch 12/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0114
Epoch 13/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0124
Epoch 14/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0091
Epoch 15/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0153
Epo

In [56]:
import numpy as np

# Extract the last 30 days of the scaled data
last_30_days = scaled_data[-5:]

# Reshape for the LSTM input: (1, timesteps, features)
X_test = np.array([last_30_days])
print(f"Input shape for prediction: {X_test.shape}")
# Predict the next day's stock prices
predicted_scaled = model.predict(xtest)

print(f"Predicted scaled prices: {predicted_scaled}")

Input shape for prediction: (1, 5, 35)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
Predicted scaled prices: [[0.6584858  0.62879133 0.60024697 0.5886026 ]
 [0.8782338  0.889524   0.82431567 0.8352408 ]
 [0.82808745 0.8154922  0.7576627  0.7536075 ]
 [0.80358434 0.7846284  0.7307929  0.72300035]
 [0.7580509  0.7308044  0.68458194 0.6722486 ]
 [0.74326813 0.7162854  0.67275    0.6609363 ]
 [0.64481926 0.5998817  0.57277244 0.551132  ]
 [0.6461569  0.60803896 0.5811525  0.56409365]
 [0.6133894  0.57070583 0.5493722  0.53000027]
 [0.67595387 0.64871144 0.6172043  0.60680664]
 [0.72007275 0.69899195 0.6599871  0.6527214 ]
 [0.72365963 0.6998651  0.66004324 0.65085393]
 [0.74045706 0.71854204 0.6758406  0.6675259 ]
 [0.77126276 0.75392294 0.7060088  0.70006406]
 [0.7060276  0.6733329  0.63606846 0.6212735 ]
 [0.7598572  0.74108213 0.69511914 0.68847066]
 [0.7581358  0.73698246 0.69117326 0.68295425]
 [0.76042116 0.7389269  0.69267946 0.684178  ]
 [0.83463955 0.82692575 0.7682875  0.767368  ]
 [0.77616346 0.75257546 0.70334786 0.693065

In [57]:
print(predicted_scaled.shape)
predicted_scaled = np.append(predicted_scaled,np.zeros((predicted_scaled.shape[0],31)),axis=1)
predicted_scaled.shape

(21, 4)


(21, 35)

In [58]:
inverse_scaled_data = scaler.inverse_transform(predicted_scaled)

In [59]:
from sklearn.metrics import mean_squared_error
ytest=np.append(ytest,np.zeros((ytest.shape[0],31)),axis=1)
print(ytest.shape)

(21, 35)


In [60]:
ytest= scaler.inverse_transform(ytest)
test_error= mean_squared_error(inverse_scaled_data[:,0:4], ytest[:,0:4])
inverse_scaled_data[:,0:4]

array([[119.56820814, 122.26102795, 115.62407557, 118.02218193],
       [127.3165238 , 130.92778269, 122.95336258, 125.80854728],
       [125.5483637 , 128.46696624, 120.77314453, 123.2313856 ],
       [124.68438392, 127.44105265, 119.8942323 , 122.26511768],
       [123.07887548, 125.65194234, 118.38267234, 120.66288547],
       [122.5576344 , 125.16933149, 117.99564962, 120.30575606],
       [119.08632695, 121.3000719 , 114.72538396, 116.83923605],
       [119.1334924 , 121.57121904, 114.99949556, 117.24843448],
       [117.97810907, 120.33026557, 113.9599619 , 116.17210674],
       [120.1841332 , 122.92317254, 116.17875023, 118.59688312],
       [121.73976505, 124.59449701, 117.57817504, 120.04641202],
       [121.86623874, 124.62352047, 117.58001163, 119.98745591],
       [122.45851592, 125.24434195, 118.0967438 , 120.51378941],
       [123.54472521, 126.42040325, 119.0835448 , 121.54101936],
       [121.24453411, 123.74159089, 116.79579672, 119.05360246],
       [123.14256418, 125

In [61]:
test_error/2

12.383384831630341

In [62]:
last_30_days = scaled_data[-5:]

# Reshape for the LSTM input: (1, timesteps, features)
X_test = np.array([last_30_days])
print(f"Input shape for prediction: {X_test.shape}")
# Predict the next day's stock prices
predicted_scaled1 = model.predict(X_test)

print(f"Predicted scaled prices: {predicted_scaled1}")

predicted_scaled1 = np.append(predicted_scaled1,np.zeros((predicted_scaled1.shape[0],31)),axis=1)

inverse_scaled_data1 = scaler.inverse_transform(predicted_scaled1)
print("Next day prediction :")
print(inverse_scaled_data1[:,0:4])

Input shape for prediction: (1, 5, 35)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step
Predicted scaled prices: [[0.84809566 0.83377236 0.77225727 0.7665558 ]]
Next day prediction :
[[126.2538531  129.07459848 121.25053241 123.64016253]]


In [63]:
predicted_scaledt = model.predict(xtrain)
predicted_scaledt = np.append(predicted_scaledt,np.zeros((predicted_scaledt.shape[0],31)),axis=1)
inverse_scaled_datat = scaler.inverse_transform(predicted_scaledt)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step


In [64]:
from sklearn.metrics import mean_squared_error
ytrain=np.append(ytrain,np.zeros((ytrain.shape[0],31)),axis=1)
print(ytrain.shape)
ytrain= scaler.inverse_transform(ytrain)
train_error= mean_squared_error(ytrain[:,0:4],inverse_scaled_datat[:,0:4])

(82, 35)


In [65]:
train_error/2

3.2350518357181306

In [66]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(ytest[:,0:4],inverse_scaled_data[:,0:4])
mape

0.03511044068638261